In [1]:
# test for cuda and cudnn
import torch
print("version", torch.__version__)
print("cuda available", torch.cuda.is_available())
print("cudnn", torch.backends.cudnn.enabled)
print("device_name", torch.cuda.get_device_name(0))
print("device count", torch.cuda.device_count())
print("current dev", torch.cuda.current_device())
print("device zero", torch.cuda.device(0))


version 2.1.2
cuda available True
cudnn True
device_name NVIDIA GeForce GTX TITAN X
device count 4
current dev 0
device zero <torch.cuda.device object at 0x7fbcc8634be0>


In [8]:
from ocatari.core import OCAtari
import random

env = OCAtari("MsPacman-v4", mode="vision", hud=True, render_mode="rgb_array")
observation, info = env.reset()
env.objects

[Player at (78, 103), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10)]

In [9]:
from ocatari.ram.extract_ram_info import detect_objects_raw, detect_objects_revised, init_objects, get_max_objects

objects = init_objects("MsPacman", True)
objects

[Player at (78, 103), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10),
 Ghost at (79, 57), (9, 10),
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## RETRIEVE LOGS

In [ ]:
### WEB scraping
import pandas as pd
import requests
from csv import reader
import os


def URLs(fold, trial):
    URLs_dict = {
    'avg_rew' : f'http://localhost:6006/data/plugin/scalars/scalars?tag=epoch_accuracy&run=fold{fold}%5C{trial}%5Cexecution0%5Ctrain&format=csv',
    'val_accuracy' : f'http://localhost:6006/data/plugin/scalars/scalars?tag=epoch_accuracy&run=fold{fold}%5C{trial}%5Cexecution0%5Cvalidation&format=csv',
    'val_loss' : f'http://localhost:6006/data/plugin/scalars/scalars?tag=epoch_loss&run=fold{fold}%5C{trial}%5Cexecution0%5Cvalidation&format=csv',
    'train_loss' : f'http://localhost:6006/data/plugin/scalars/scalars?tag=epoch_loss&run=fold{fold}%5C{trial}%5Cexecution0%5Ctrain&format=csv'
    }
    return URLs_dict

def tb_data(log_dir, mode, fold, num_trials):
    trials = os.listdir(log_dir)
    fdf = {}
    for i, trial in enumerate(trials[:num_trials]):
        r = requests.get(URLs(fold, trial)[mode], allow_redirects=True)
        data = r.text
        data_csv = reader(data.splitlines())
        data_csv = list(data_csv)
        df = pd.DataFrame(data_csv)
        headers = df.iloc[0]
        df  = pd.DataFrame(df.values[1:], columns=headers)
        if i == 0:
            fdf['Step'] = df['Step']  
        fdf[f'trial {trial}'] = df['Value']
    fdf = pd.DataFrame(fdf)
    return fdf

In [ ]:
#### Tensorboard Scalar Plotting #1
import tensorflow as tf
import numpy as np

TAG_NAME = "classification_loss"

writer_train = tf.summary.create_file_writer("train")

for epoch in range(200):
    data = np.math.exp(-(epoch/30)) + 0.1*np.random.random()
    with writer_train.as_default():
        tf.summary.scalar(name=TAG_NAME, data=data, step=epoch)


import tensorflow as tf
import matplotlib.pyplot as plt

TAG_NAME = "classification_loss"
tf_event_file = "train/events.out.tfevents.1618103384.eafddbceac44.283.2075165.v2"

value_list = []
for e in tf.compat.v1.train.summary_iterator(tf_event_file):
    for v in e.summary.value:
        if v.tag == TAG_NAME:
            value = tf.make_ndarray(v.tensor)
            value_list.append(value)

plt.plot(value_list)
plt.grid()
plt.show()

In [ ]:
#### Tensorboard Scalar Plotting #2

import tensorflow as tf
import matplotlib.pyplot as plt

def get_scalar_run_tensorboard(tag, filepath):
    values,steps = [],[]
    for e in tf.compat.v1.train.summary_iterator(filepath):
        if len(e.summary.value)>0: #Skip first empty element
            if e.summary.value[0].tag==tag:
                tensor = (e.summary.value[0].tensor)
                value,step = (tf.io.decode_raw(tensor.tensor_content,tf.float32)[0].numpy(),e.step)
                values.append(value)
                steps.append(step)
    return values,steps

val,st = get_scalar_run_tensorboard("epoch_loss",
                               "./logs/your_log/your_run/train/your_file.v2")

import matplotlib.pyplot as plt
plt.plot(st,val)